In [18]:
import pandas as pd
from xgboost import XGBClassifier, plot_importance
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import numpy as np
from matplotlib import pyplot
from sklearn.cluster import KMeans, MeanShift

df = pd.read_csv("/mnt/c/users/gautam/source/repos/VoltInsightsAutomation/Reports/data.csv")
original_df = pd.DataFrame.copy(df)

unused_columns = ['Crew Type Description', 'Crew Number', 'Productive Revenue per Manhour', 
                  'Billable Hours', 'Productive Revenue', 'Total Crew Cost Less Externals',
                  'Billable Hours per Resource']

for col in df.columns:
    if col not in unused_columns and (df[col].dtype != np.int64 and df[col].dtype != np.float64):
            df[col] = df[col].fillna('0')
            df[col] = df[col].str.replace('$', '')
            df[col] = df[col].str.replace('%', '')
            df[col] = df[col].str.replace('(', '')
            df[col] = df[col].str.replace(')', '')
            df[col] = df[col].astype(float)
    else:
        df.drop(col, 1, inplace=True)

def margin_range(row):
    if row['Crew Margin Less Externals per Direct Manhour'] > 35:
        val = 3 #'Greater than 35'
    elif row['Crew Margin Less Externals per Direct Manhour'] >= 10 and row['Crew Margin Less Externals per Direct Manhour'] <=35:
        val = 0 #'10 to 35'
    else:
        val = 0 #'Less than 10'
    return val

df['Crew Margin'] = df.apply(margin_range, axis=1)
df.drop('Crew Margin Less Externals per Direct Manhour', 1, inplace=True)


In [19]:
def handle_non_numerical_data(df):
    columns = df.columns.values

    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]
        
        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x += 1
            
            df[column] = list(map(convert_to_int, df[column]))
    return df
df = handle_non_numerical_data(df)
#df.drop(['ticket'], 1, inplace=True)
#df.drop(['ticket','home.dest'], 1, inplace=True)


In [20]:
X = np.array(df.drop(['Crew Margin'], 1).astype(float))
X = preprocessing.scale(X)
y = np.array(df['Crew Margin'])

clf = MeanShift()
clf.fit(X)

labels = clf.labels_

original_df['cluster_group'] = np.nan

for i in range(len(X)):
    original_df['cluster_group'].iloc[i] = labels[i]

In [23]:
original_df.head()

n_clusters_ = len(np.unique(labels))
print(n_clusters_)

5
